In [1]:
#folds_comparison

import os
import re
import pandas as pd
from pathlib import Path

def parse_classification_report(file_path):
    """Parse classification_report.txt file and extract key metrics"""
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Extract accuracy
    accuracy_match = re.search(r'accuracy\s+(\d+\.\d+)', content)
    accuracy = float(accuracy_match.group(1)) if accuracy_match else None
    
    # Extract macro avg precision, recall, and f1-score
    macro_avg_match = re.search(r'macro avg\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)', content)
    macro_precision = float(macro_avg_match.group(1)) if macro_avg_match else None
    macro_recall = float(macro_avg_match.group(2)) if macro_avg_match else None
    macro_f1 = float(macro_avg_match.group(3)) if macro_avg_match else None
    
    return accuracy, macro_precision, macro_recall, macro_f1


def compare_folds(base_path=".", model_name="YourModel"):
    """Compare results across all folds"""
    results = []
    
    # Process each fold
    for fold_num in range(1, 11):  # fold_1 to fold_10
        fold_dir = Path(base_path) / f"fold_{fold_num}"
        report_file = fold_dir / "classification_report.txt"
        
        if report_file.exists():
            try:
                accuracy, macro_precision, macro_recall, macro_f1 = parse_classification_report(report_file)

                # Append fold result to the list
                results.append({
                    'model_name': model_name,
                    'fold_number': fold_num,
                    'accuracy': accuracy,
                    'precision_macro_avg': macro_precision,
                    'recall_macro_avg': macro_recall,
                    'f1_score_macro_avg': macro_f1
                })
                
            except Exception as e:
                print(f"Error processing fold_{fold_num}: {e}")
        else:
            print(f"Warning: {report_file} not found")
    
    # Process test_evaluation folder
    test_dir = Path(base_path) / "test_evaluation"
    test_report_file = test_dir / "classification_report.txt"
    
    if test_report_file.exists():
        try:
            accuracy, macro_precision, macro_recall, macro_f1 = parse_classification_report(test_report_file)
            
            results.append({
                'model_name': model_name,
                'fold_number': 'Test',
                'accuracy': accuracy,
                'precision_macro_avg': macro_precision,
                'recall_macro_avg': macro_recall,
                'f1_score_macro_avg': macro_f1
            })
            
            print(f"Processed test_evaluation: Accuracy={accuracy:.4f}, Precision={macro_precision:.4f}, Recall={macro_recall:.4f}, Macro F1={macro_f1:.4f}")
            
        except Exception as e:
            print(f"Error processing test_evaluation: {e}")
    else:
        print(f"Warning: {test_report_file} not found")

    
    return results

def generate_comparison_report(results, model_name):
    """Generate comparison report and statistics"""
    if not results:
        print("No results to analyze")
        return
    
    df = pd.DataFrame(results)
    
    # Separate test results from fold results
    fold_results = df[df['fold_number'] != 'Test']
    test_results = df[df['fold_number'] == 'Test']
    
    # Calculate averages and add average row (only from fold results)
    if len(fold_results) > 0:
        avg_accuracy = fold_results['accuracy'].mean()
        avg_precision = fold_results['precision_macro_avg'].mean()
        avg_recall = fold_results['recall_macro_avg'].mean()
        avg_f1_macro = fold_results['f1_score_macro_avg'].mean()
        
        avg_row = {
            'model_name': model_name,
            'fold_number': 'Avg',
            'accuracy': avg_accuracy,
            'precision_macro_avg': avg_precision,
            'recall_macro_avg': avg_recall,
            'f1_score_macro_avg': avg_f1_macro
        }
        
        # Reconstruct dataframe: folds + avg + test
        df = pd.concat([
            fold_results, 
            pd.DataFrame([avg_row]), 
            test_results
        ], ignore_index=True)

    # Sort the dataframe for clarity
    df.sort_values(by=['model_name', 'fold_number'], ascending=[True, True], inplace=True)

    # Save results to CSV
    output_file = "fold_comparison_results.csv"

    # Check if the file already exists
    if not os.path.exists(output_file):
        # If file doesn't exist, write the header
        df.to_csv(output_file, index=False, float_format='%.4f')
        print(f"\nResults saved to: {output_file}")
    else:
        # If file exists, append without writing the header
        df.to_csv(output_file, index=False, header=False, mode='a', float_format='%.4f')
        print(f"\nResults appended to: {output_file}")
    
    return df


def main():
    # Configuration
    models = ["folds data"]  # List of models to process
    
    for model_name in models:
        BASE_PATH = f"./{model_name}"  # Adjust the path to the model folder

        print(f"\nStarting fold comparison analysis for {model_name}...")
        
        # Parse all fold results
        results = compare_folds(BASE_PATH, model_name)
        
        if results:
            # Generate comparison report
            df = generate_comparison_report(results, model_name)

        else:
            print(f"No valid results found for {model_name}. Please check your file paths and formats.")

if __name__ == "__main__":
    main()



Starting fold comparison analysis for folds data...

Results saved to: fold_comparison_results.csv
